### Auxiliar Functions

In [25]:
def print_red(text):
    print('\x1b[31m' + text + '\x1b[0m')
    
def print_yellow(text):
    print('\x1b[33m' + text + '\x1b[0m')

def print_green(text):
    print('\x1b[32m' + text + '\x1b[0m')

def print_pink(text):
    print('\x1b[35m' + text + '\x1b[0m')

def print_cyan(text):
    print('\x1b[36m' + text + '\x1b[0m')

## Get the Instances / Ingredient list
The instances have been generated using the notebook: Generate Instance.ipynb

In [26]:
import os
import re

instances_folder = '..\instances'
instances_path = [os.path.join(instances_folder, f) for f in os.listdir(instances_folder) if f.endswith('.dzn')]
# Sort
def numeric_sort_key(file_path):
    match = re.search(r'\d+', file_path)
    return int(match.group()) if match else file_path

instances_path = sorted(instances_path, key=numeric_sort_key)
# print(instances_path)

# Minizinc
### Chosen model with annotations
The models have been generated using the notebook: GenerateAnnotations.ipynb

In [27]:
model = '../models/winners/smallest-indomain.mzn'

### Solvers

In [28]:
!minizinc --solvers
solvers = ["HiGHS", "COIN-BC", "OR-TOOLS"]

MiniZinc driver.
Available solver configurations:
  Chuffed 0.11.0 (org.chuffed.chuffed, cp, lcg, int)
  COIN-BC 2.10.8/1.17.7 (org.minizinc.mip.coin-bc, mip, float, api, osicbc, coinbc, cbc)
  CPLEX <unknown version> (org.minizinc.mip.cplex, mip, float, api)
  findMUS 0.7.0 (org.minizinc.findmus)
  Gecode 6.3.0 (org.gecode.gecode, default solver, cp, int, float, set, restart)
  Gecode Gist 6.3.0 (org.gecode.gist, cp, int, float, set, restart)
  Globalizer 0.1.7.2 (org.minizinc.globalizer, experimental, tool)
  Gurobi <unknown version> (org.minizinc.mip.gurobi, mip, float, api)
  HiGHS 1.4.2 (org.minizinc.mip.highs, mip, float, api, highs)
  OR-TOOLS 9.6.0 (com.google.or-tools)
  SCIP <unknown version> (org.minizinc.mip.scip, mip, float, api)
  Xpress <unknown version> (org.minizinc.mip.xpress, mip, float, api)
Search path for solver configurations:
  C:\Users\juanj\AppData\Roaming/MiniZinc/solvers
  C:\Program Files\MiniZinc\share\minizinc/solvers


## Calculate labels

In [29]:
import random
import subprocess
# Returns the solver who solved the model in the best way possible
def solve_with_minizinc(instances, timeout_mzn, timeout):    
    winner = [0] * len(instances) # 0 for HiGHS, 1 for COIN-BC
    timeout_mzn = timeout_mzn*1000
    
    for i, instance in enumerate(instances):
        mnt = float('inf')
        time = float('inf')
        for j, solver in enumerate(solvers):
            # Run the minizinc command for the current model and data file  
            cmd = f"minizinc --solver {solver} --output-time {model} {instance} --output-time --solver-time-limit {timeout_mzn}" 
            proc = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE)
            try: 
                # Get output from fzn
                stdout, stderr = proc.communicate(timeout=timeout) 
                stdout = stdout.decode()
                try:
                    # Store value mnt (temp).
                    start = stdout.find('mnt = ')+6
                    end = stdout.find(';',start)
                    mnt_temp = int(stdout[start:end])
                    # Store value time (temp).
                    time_temp = float(stdout.split(' ')[-2])
                except:
                    mnt_temp = float('inf')
                    time_temp = float('inf')
                print_green(f'{solver} with {instance} SOLVED with a value of {mnt_temp} in {time_temp:.2f} seconds.')

            except subprocess.TimeoutExpired: 
                print_red(f'Error from minizinc: Stopping.')
                mnt_temp = float('inf')
                time_temp = float('inf')
                if os.name == 'nt': # If the os is Windows
                    subprocess.call(['taskkill', '/F', '/T', '/PID', str(proc.pid)]) # Force kill the process
                else:
                    os.killpg(os.getpgid(proc.pid), signal.SIGTERM) # Kill the process group

            if mnt >= mnt_temp: # If the value is minor from the stored
                if mnt == mnt_temp: # If the value is equal from the stored compare time
                    if time >= time_temp: # If the value of time from minizinc is minor from the stored
                        if time == time_temp: # If the value of time from minizinc is equal from the stored choose random
                            winner[i] = random.randint(j-1,j)    
                        else:
                            mnt = mnt_temp
                            time = time_temp
                            winner[i] = j
                else:
                    mnt = mnt_temp
                    time = time_temp
                    winner[i] = j
        print_yellow(str(winner[i])+" WON!")
    return winner

In [30]:
# OG : labels = solve_with_minizinc(instances_path, 10, 15)
labels = solve_with_minizinc(instances_path, 10, 15)

HiGHS with ..\instances\0.dzn SOLVED with a value of 30 in 1.88 seconds.
COIN-BC with ..\instances\0.dzn SOLVED with a value of 310 in 10.54 seconds.
OR-TOOLS with ..\instances\0.dzn SOLVED with a value of 3340 in 10.20 seconds.
0 WON!
HiGHS with ..\instances\1.dzn SOLVED with a value of 40 in 3.18 seconds.
COIN-BC with ..\instances\1.dzn SOLVED with a value of 50 in 10.23 seconds.
OR-TOOLS with ..\instances\1.dzn SOLVED with a value of 2840 in 10.24 seconds.
0 WON!
HiGHS with ..\instances\2.dzn SOLVED with a value of 30 in 6.37 seconds.
COIN-BC with ..\instances\2.dzn SOLVED with a value of 120 in 10.34 seconds.
OR-TOOLS with ..\instances\2.dzn SOLVED with a value of 8210 in 10.27 seconds.
0 WON!
HiGHS with ..\instances\3.dzn SOLVED with a value of 30 in 7.63 seconds.
COIN-BC with ..\instances\3.dzn SOLVED with a value of 30 in 4.38 seconds.
OR-TOOLS with ..\instances\3.dzn SOLVED with a value of 120 in 10.28 seconds.
1 WON!
HiGHS with ..\instances\4.dzn SOLVED with a value of 50 in 1

In [31]:
path = '../instances/labels.txt'
with open(path, 'w', encoding='utf8') as file_object:
    file_object.write(str(labels))